In [29]:
%run "sql.ipynb"
import math
from scipy.stats import sem
from scipy import stats
x_sist = 0.5
alpha = 0.95
avg_x = {}
sigma = {}
SEM = {}
x_sl = {}
abs_x = {}
otn_x = {}
for i in dict_lab1:
    n = len(dict_lab1[i])
    avg_x[i] = np.mean(dict_lab1[i])
    sigma[i] = np.std(dict_lab1[i], ddof=1)
    SEM[i] = np.std(dict_lab1[i], ddof=1) / np.sqrt(np.size(dict_lab1[i]))
    x_sl[i] = stats.t.ppf((1 + alpha)/2, n-1) * SEM[i]
    abs_x[i] = math.sqrt(x_sist ** 2 + x_sl[i] ** 2)
    otn_x[i] = abs_x[i]/avg_x[i]


Connection to PostgreSQL DB successful


In [30]:
import sympy as sym
dx = []
derivative_f = []

for i in range(len(val_names)):
    dx.append(sym.Symbol(val_names[i]))

avg_f = avg_x[val_names[0]] * avg_x[val_names[1]] * avg_x[val_names[2]]

f = pow(dx[0],1) * pow(dx[1],1) * pow(dx[2],1)

dfdx = []
for i in range(len(val_names)):
    df = f.diff(dx[i])
    derivative_f.append(df)
    f_i = sym.lambdify([j for j in dx], df)
    dfdx.append(f_i(avg_x['a'], avg_x['b'], avg_x['c']))

In [35]:
abs_f = 0
for i, v in enumerate(dict_lab1):
    abs_f += (dfdx[i] * abs_x[v]) ** 2
abs_f = math.sqrt(abs_f)
otn_f = abs_f/avg_f

In [38]:
connection = create_connection("postgres", "postgres", "1234", "localhost", "5432")
for i in dict_lab1:
    update = f"""
UPDATE results_lab1
SET
avg_x = '{avg_x[i]}',
abs_x = '{abs_x[i]}',
otn_x = '{otn_x[i]}',
s = '{SEM[i]}'
WHERE
value = '{i}'
"""
    update_table = execute_query(connection, update)

update = f"""
UPDATE results_lab1
SET
avg_x = '{avg_f}',
abs_x = '{abs_f}',
otn_x = '{otn_f}'
WHERE
value = 'V'
"""
update_table = execute_query(connection, update)

Connection to PostgreSQL DB successful
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
